In [1]:
import datasets

/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [45]:
data_path = '/data/kcl/lpy/data/OpenThoughts'
cache_path = '/data/kcl/lpy/hf'
dataset = datasets.load_dataset(data_path, "metadata", cache_dir=cache_path, split="train")

In [65]:
temp = dataset.to_pandas()

In [69]:
temp['source'].value_counts()

source
numina_math          89120
taco                  6983
code_contests         6510
apps                  4794
codeforces            1617
camelai_biology       1246
camelai_physics       1246
camelai_chemistry     1222
riddle_sense          1219
Name: count, dtype: int64

In [68]:
temp.head()

,problem,deepseek_reasoning,deepseek_solution,ground_truth_solution,domain,source,test_cases,starter_code
0,"Let \( a, b, c \) be positive real numbers. Pr...","Okay, so I need to prove this inequality: \(\f...",To prove the inequality \n\n\[\n\frac{1}{a(1+b...,1. Consider the given inequality:\n\n\[\n\frac...,math,numina_math,NaN,NaN
1,"Given real numbers \( a, b, c \) and a positiv...","Okay, let's try to tackle this problem step by...","Given real numbers \(a, b, c\) and a positive ...",\nWe begin by analyzing the function \( f(x) =...,math,numina_math,NaN,NaN
2,"In triangle $ABC$, $CA = CB$, and $D$ is the m...","Okay, so I have this geometry problem here. Le...","In triangle \(ABC\), \(CA = CB\), making it is...",1. **Identify Key Elements**: Consider \( \tri...,math,numina_math,NaN,NaN
3,"Let \( p = 2^{3009}, q = 3^{2006}, \) and \( r...","Okay, so I need to figure out which of the giv...",To determine which of the given statements is ...,"Given the values:\n\[ p = 2^{3009}, \quad q = ...",math,numina_math,NaN,NaN
4,"When \( a < -1 \), the nature of the roots for...","Okay, so I need to figure out the nature of th...",To determine the nature of the roots for the q...,\nGiven the quadratic equation:\n\n\[\n(a^3 + ...,math,numina_math,NaN,NaN


In [58]:
print(dataset['problem'][0])

Let \( a, b, c \) be positive real numbers. Prove that

$$
\frac{1}{a(1+b)}+\frac{1}{b(1+c)}+\frac{1}{c(1+a)} \geq \frac{3}{1+abc},
$$

and that equality occurs if and only if \( a = b = c = 1 \).


In [71]:
print(dataset['ground_truth_solution'][1])

Given real numbers \(a, b, c\) and a positive number \(\lambda\) such that the polynomial \(f(x) = x^3 + ax^2 + bx + c\) has three real roots \(x_1, x_2, x_3\) with the conditions \(x_2 - x_1 = \lambda\) and \(x_3 > \frac{1}{2}(x_1 + x_2)\), we need to find the maximum value of \(\frac{2a^3 + 27c - 9ab}{\lambda^3}\).

Using Vieta's formulas for the polynomial \(f(x)\):
1. \(x_1 + x_2 + x_3 = -a\)
2. \(x_1x_2 + x_1x_3 + x_2x_3 = b\)
3. \(x_1x_2x_3 = -c\)

We parametrize the roots as follows:
- \(x_1 = t - \frac{\lambda}{2}\)
- \(x_2 = t + \frac{\lambda}{2}\)
- \(x_3 = t + s\) where \(s > 0\)

Substituting these into Vieta's formulas, we get:
- \(S = x_1 + x_2 + x_3 = 3t + s\)
- \(P = x_1x_2 + x_1x_3 + x_2x_3 = 3t^2 + 2ts - \frac{\lambda^2}{4}\)
- \(Q = x_1x_2x_3 = (t^2 - \frac{\lambda^2}{4})(t + s)\)

Substituting \(a = -S\), \(b = P\), and \(c = -Q\) into the expression \(2a^3 + 27c - 9ab\), we simplify it to:
\[
-2s^3 + \frac{9}{2}\lambda^2 s
\]

Introducing \(k = \frac{s}{\lambda}\),

In [33]:
    data_source = "open-thoughts/OpenThoughts-114k"
    instruction_following = 'Let\'s think step by step and output the final answer after "####".'
    def make_map_fn(split):
        def process_fn(example, idx):
            question_raw = example.pop("problem")

            question = question_raw + " " + instruction_following

            answer_raw = example.pop("ground_truth_solution")
            solution = answer_raw
            data = {
                "data_source": data_source,
                "prompt": [
                    {
                        "role": "user",
                        "content": question,
                    }
                ],
                "ability": "math",
                "reward_model": {"style": "rule", "ground_truth": solution},
                "extra_info": {
                    "split": 'train',
                    "index": idx,
                    "answer": answer_raw,
                    "question": question_raw,
                },
            }
            return data

        return process_fn

    dataset = dataset.map(function=make_map_fn('train'), with_indices=True)

Map: 100%|██████████| 113957/113957 [00:21<00:00, 5374.89 examples/s]


In [36]:
local_dir = '/data/kcl/lpy/data/OpenThoughts'
import os
dataset.to_parquet(os.path.join(local_dir, "train_verl.parquet"))

Creating parquet from Arrow format: 100%|██████████| 53/53 [00:27<00:00,  1.95ba/s]


5241699449

In [42]:
print(dataset['reward_model'][0]['ground_truth'])

1. Consider the given inequality:

\[
\frac{1}{a(1+b)}+ \frac{1}{b(1+c)}+ \frac{1}{c(1+a)} \geq \frac{3}{1 + abc}
\]

To simplify, we add \( \frac{3}{1 + abc} \) to both sides. The new inequality becomes:

\[
\frac{1}{a(1+b)} + \frac{1}{b(1+c)} + \frac{1}{c(1+a)} + \frac{3}{1 + abc} \geq \frac{6}{1 + abc}
\]

2. Let's analyze each term with an added \( \frac{1}{1 + abc} \):

\[
\frac{1}{a(1+b)} + \frac{1}{1 + abc}, \quad \frac{1}{b(1+c)} + \frac{1}{1 + abc}, \quad \frac{1}{c(1+a)} + \frac{1}{1 + abc}
\]

We can rewrite them as follows:

\[
\begin{aligned}
\frac{1}{a(1+b)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{a(1+b)} + 1 \right), \\
\frac{1}{b(1+c)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{b(1+c)} + 1 \right), \\
\frac{1}{c(1+a)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{c(1+a)} + 1 \right).
\end{aligned}
\]

3. Simplifying inside the parentheses:

\[
\begin{aligned}
\frac{1 + abc}{a(1+b)} + 1 &= \frac{1 + abc + a(1+b)